# Corporate Decarbonization Research
---

## Import modules

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import plotly as pltly

---

## Data Cleaning and Importing

In this section of the notebbook, I will be importing necessary modules and cleaning/standardizing our datasets.

### Import modules and libraries

In [2]:
foodag = pd.read_csv('data/food&ag.csv')
energy = pd.read_csv('data/energy.csv')
auto = pd.read_csv('data/auto.csv')
tech = pd.read_csv('data/tech.csv')


### Clean the Data

In [24]:
# Add sector columns to concatenate df's

foodag['SECTOR'] = ['Food & Agriculture'] * len(foodag['COMPANY NAME'])
energy['SECTOR'] = ['Energy'] * len(energy['COMPANY NAME'])
auto['SECTOR'] = ['Auto'] * len(auto['COMPANY NAME'])
tech['SECTOR'] = ['Tech'] * len(tech['COMPANY NAME'])
foodag.shape, energy.shape, auto.shape, tech.shape


((39, 10), (52, 10), (67, 11), (34, 10))

In [25]:
# clean tech to standardize columns across all df's

tech = tech.rename(columns={'CARBON INTENSITY\n(Scope 1 & 2 g CO2e/ $ Sales) \ncalculated' :
'CARBON INTENSITY\n(Scope 1 & 2 g CO2e / $ Sales)'})

In [26]:
# concatenate dataframes together

sectors = pd.concat([foodag, energy, auto, tech])
sectors = sectors.drop(columns='SCOPE 1 + SCOPE 2 EMISSIONS')
sectors.head()

,COMPANY NAME,YEAR,SCOPE 1,SCOPE 2 (location-based),SCOPE 3,TOTAL EMISSIONS,TOTAL REVENUE (miillion $),CARBON INTENSITY\n(Scope 1 & 2 g CO2e / $ Sales),COUNTRY OF ORIGIN,SECTOR
0,Anheuser-Busch InBev,2018,3700760.0,2383505.0,24431839.0,30516104.0,54619.0,111.39,Belgium,Food & Agriculture
1,Anheuser-Busch InBev,2019,3600710.0,2215902.0,26484447.0,32301059.0,52329.0,111.15,Belgium,Food & Agriculture
2,Anheuser-Busch InBev,2020,3081905.0,2227758.0,23652933.0,28962596.0,46881.0,113.26,Belgium,Food & Agriculture
3,Anheuser-Busch InBev,2021,2953896.0,2243990.0,26527444.0,31725330.0,54304.0,95.72,Belgium,Food & Agriculture
4,Danone,2018,753461.0,959554.0,24885804.0,26598819.0,24651.0,69.49,France,Food & Agriculture


---

## Exploratory Data Analysis (EDA)

In [63]:
sectors.groupby('SECTOR').mean(numeric_only=True).drop(columns=['YEAR']).round(0).sort_values(by='TOTAL EMISSIONS', ascending=False)

,SCOPE 1,SCOPE 2 (location-based),SCOPE 3,TOTAL EMISSIONS,TOTAL REVENUE (miillion $)
SECTOR,,,,,
Energy,49572817.0,7985842.0,429716132.0,390728248.0,120495.0
Auto,1149196.0,2299911.0,190202075.0,193927442.0,118852.0
Food & Agriculture,1632704.0,1226111.0,41175943.0,44034758.0,44966.0
Tech,2495589.0,5359486.0,27581036.0,35436111.0,213245.0


In [64]:
country = sectors.groupby('COUNTRY OF ORIGIN').mean(numeric_only=True).drop(columns=['YEAR']).round(0).sort_values(by='TOTAL EMISSIONS', ascending=False)
country

,SCOPE 1,SCOPE 2 (location-based),SCOPE 3,TOTAL EMISSIONS,TOTAL REVENUE (miillion $)
COUNTRY OF ORIGIN,,,,,
"United Kingdom, Netherlands",66000000.0,10500000.0,1.019486e+09,1.095986e+09,301128.0
Russsia,72418092.0,13464851.0,7.015146e+08,9.397628e+08,NaN
United Kingdom,43225000.0,4175000.0,3.560667e+08,3.144500e+08,214907.0
Norway,13742474.0,189524.0,2.782205e+08,2.921525e+08,70173.0
"Italy, United States",1227754.0,2414369.0,2.668018e+08,2.706001e+08,136563.0
Germany,2094966.0,2623166.0,2.120192e+08,2.169266e+08,183161.0
Japan,795872.0,1976986.0,1.866242e+08,1.893971e+08,160869.0
United States,13699449.0,3512779.0,1.517251e+08,1.584735e+08,150594.0
China,25552117.0,9301497.0,7.942017e+07,1.236736e+08,19835.0
